In [9]:
import pandas as pd
from ltp import LTP
import re
ltp = LTP()

In [10]:
with open("professions.txt",'r' , encoding='UTF-8') as f:
    profession = f.read()
    profession = profession.split('\n')

def clean (text): # 洗掉英文字符
    text =  str(text)
    temp = re.sub('[a-zA-Z]','',text)
    return temp

def process (text):
    seg, hidden = ltp.seg([ text])
    ner = ltp.ner(hidden)
    
    name = []
    age = []
    place = []
    action = []
    work = []
    p1 = r"男|女"
    p2 = r'岁'
    age = re.findall( p2, text)
    gender = re.findall( p1, text)
    for i in profession:
        if i in text and i != '员':
            work.append(i)
    
    # 正则表达式 提取 年龄 性别
    # 命名实体nh 姓名 ni机构 ns 地点
    # ner[0] 结构为 （标签，标记起始，标记末尾）
    for item in ner[0]:
        tag, start, end = item
        #print(tag,":", "".join(seg[0][start:end + 1]))
        if(tag == 'Nh'): #name 
            name.append("".join(seg[0][start:end + 1]) )
        elif ( tag == 'Ns' or  tag == 'Ni') :
            place.append("".join(seg[0][start:end + 1]) )
    
    # pos 标记动作
    ## 动作标记为'v'
    
    pos = ltp.pos(hidden)
    for i in range(len(seg[0])):
        if (pos[0][i] == 'v'):
            action.append(seg[0][i])


    # 数据0 1 标记
    #label
    if len(name) == 0:
        name_label = 0
    elif '某' in name:
        name_label = 0.5
    else:
        name_label = 1
        
    age_label = 1 if len(age) != 0 else 0
    gender_label = 1 if len(gender) != 0 else 0
    work_label = 1 if len(work) != 0 else 0
    
    return (name ,name_label, age, age_label ,gender, gender_label,  work, work_label ,place, action )

In [11]:
#打开excel
import pandas as pd
df = pd.read_excel('./Data/扬州.xlsx', engine='openpyxl', header = 0)
df.head(5)

,标题,标题链接,时间,文本2
0,9月13日扬州新冠肺炎疫情情况通报,http://www.yangzhou.gov.cn/yzfy/yzyqtg/202109/...,[2021-09-14],江苏省卫健委9月14日发布，9月13日0-24时，江苏无新增本土确诊病例。新增出院病例3例（...
1,9月12日扬州新冠肺炎疫情情况通报,http://www.yangzhou.gov.cn/yzfy/yzyqtg/202109/...,[2021-09-13],江苏省卫健委9月13日发布，9月12日0-24时，江苏无新增本土确诊病例。新增出院病例1例（...
2,9月11日扬州新冠肺炎疫情情况通报,http://www.yangzhou.gov.cn/yzfy/yzyqtg/202109/...,[2021-09-12],江苏省卫健委9月12日发布，9月11日0-24时，江苏无新增本土确诊病例。新增境外输入无症状...
3,9月10日扬州新冠肺炎疫情情况通报,http://www.yangzhou.gov.cn/yzfy/yzyqtg/202109/...,[2021-09-11],江苏省卫健委9月11日发布，9月10日0-24时，江苏无新增本土确诊病例。新增出院病例4例（...
4,9月9日扬州新冠肺炎疫情情况通报,http://www.yangzhou.gov.cn/yzfy/yzyqtg/202109/...,[2021-09-10],江苏省卫健委9月10日发布，9月9日0-24时，江苏无新增本土确诊病例。新增境外输入确诊病例...


In [12]:
df['文本2'] = df['文本2'].apply(clean) #清除字母

In [13]:
# clean_data_df: [time,content]
clean_data = {'time':df['时间'] ,'content': df['文本2']}
clean_data_df = pd.DataFrame(clean_data)
clean_data_df = clean_data_df.groupby(by='time').sum('content') # 按时间合并
print(clean_data_df.head)
clean_data_df.to_excel("yz_clean.xlsx")
# df.to_csv('shenz_clean.csv', encoding='utf-8')

<bound method NDFrame.head of                                                         content
time                                                           
[2020-01-27]  截至2020年1月27日，省卫健委先后发布我市共四例新型冠状病毒感染的肺炎确诊病例。\n患者...
[2020-01-28]  1月28日，省卫健委发布了我市第五例新型冠状病毒感染的肺炎确诊病例。\n患者为男性，26岁，...
[2020-01-29]  1月29日，省卫健委新发布我市三例新型冠状病毒感染的肺炎确诊病例。\n患者曹某，女性，47岁...
[2020-02-01]  2月1日，省卫健委新发布我市五例新型冠状病毒感染的肺炎确诊病例。\n患者余某，女性，51岁，...
[2020-02-03]  2月3日，省卫健委新发布我市一例新型冠状病毒感染的肺炎确诊病例。\n患者曹某，女性，23岁，...
...                                                         ...
[2021-09-10]  江苏省卫健委9月10日发布，9月9日0-24时，江苏无新增本土确诊病例。新增境外输入确诊病例...
[2021-09-11]  江苏省卫健委9月11日发布，9月10日0-24时，江苏无新增本土确诊病例。新增出院病例4例（...
[2021-09-12]  江苏省卫健委9月12日发布，9月11日0-24时，江苏无新增本土确诊病例。新增境外输入无症状...
[2021-09-13]  江苏省卫健委9月13日发布，9月12日0-24时，江苏无新增本土确诊病例。新增出院病例1例（...
[2021-09-14]  江苏省卫健委9月14日发布，9月13日0-24时，江苏无新增本土确诊病例。新增出院病例3例（...

[73 rows x 1 columns]>


In [14]:
clean_data_df = pd.read_excel('yz_clean.xlsx', engine='openpyxl', header = 0)
print(clean_data_df.head)

<bound method NDFrame.head of             time                                            content
0   [2020-01-27]  截至2020年1月27日，省卫健委先后发布我市共四例新型冠状病毒感染的肺炎确诊病例。\n患者...
1   [2020-01-28]  1月28日，省卫健委发布了我市第五例新型冠状病毒感染的肺炎确诊病例。\n患者为男性，26岁，...
2   [2020-01-29]  1月29日，省卫健委新发布我市三例新型冠状病毒感染的肺炎确诊病例。\n患者曹某，女性，47岁...
3   [2020-02-01]  2月1日，省卫健委新发布我市五例新型冠状病毒感染的肺炎确诊病例。\n患者余某，女性，51岁，...
4   [2020-02-03]  2月3日，省卫健委新发布我市一例新型冠状病毒感染的肺炎确诊病例。\n患者曹某，女性，23岁，...
..           ...                                                ...
68  [2021-09-10]  江苏省卫健委9月10日发布，9月9日0-24时，江苏无新增本土确诊病例。新增境外输入确诊病例...
69  [2021-09-11]  江苏省卫健委9月11日发布，9月10日0-24时，江苏无新增本土确诊病例。新增出院病例4例（...
70  [2021-09-12]  江苏省卫健委9月12日发布，9月11日0-24时，江苏无新增本土确诊病例。新增境外输入无症状...
71  [2021-09-13]  江苏省卫健委9月13日发布，9月12日0-24时，江苏无新增本土确诊病例。新增出院病例1例（...
72  [2021-09-14]  江苏省卫健委9月14日发布，9月13日0-24时，江苏无新增本土确诊病例。新增出院病例3例（...

[73 rows x 2 columns]>


In [15]:
# (name ,name_label, age, age_label ,gender, gender_label,  work, work_label ,place, action )

In [16]:
name_col = []
name_label_col = []
age_col = []
age_label_col = []
gender_col = []
gender_label_col = []
work_col = []
work_label_col= []
place_col = []
action_col = []


In [17]:
for i in clean_data_df['content']:
    name ,name_label, age, age_label ,gender, gender_label,  work, work_label ,place, action = process(i)
    name_col .append(name)
    name_label_col .append(name_label)
    age_col.append(age)
    age_label_col .append(age_label)
    gender_col.append(gender)
    gender_label_col .append(gender_label)
    work_col.append(work)
    work_label_col.append(work_label)
    place_col .append(place)
    action_col .append(action)

In [18]:
result_data = {    
            'time' : clean_data_df['time'] , 
            'content': clean_data_df['content'] , 
            'name': name_col,  
            'name_label': name_label_col ,
            'age': age_col,
            'age_label': age_label_col,
            'gender':gender_col,
            'gender_label':gender_label_col ,
            'work': work_col,
            'work_label':work_label_col,
            'place':place_col,
            'action':action_col }
res = pd.DataFrame(result_data)
print(res.head)
res.to_excel("./Results/yz_res.xlsx")

<bound method NDFrame.head of             time                                            content  \
0   [2020-01-27]  截至2020年1月27日，省卫健委先后发布我市共四例新型冠状病毒感染的肺炎确诊病例。\n患者...   
1   [2020-01-28]  1月28日，省卫健委发布了我市第五例新型冠状病毒感染的肺炎确诊病例。\n患者为男性，26岁，...   
2   [2020-01-29]  1月29日，省卫健委新发布我市三例新型冠状病毒感染的肺炎确诊病例。\n患者曹某，女性，47岁...   
3   [2020-02-01]  2月1日，省卫健委新发布我市五例新型冠状病毒感染的肺炎确诊病例。\n患者余某，女性，51岁，...   
4   [2020-02-03]  2月3日，省卫健委新发布我市一例新型冠状病毒感染的肺炎确诊病例。\n患者曹某，女性，23岁，...   
..           ...                                                ...   
68  [2021-09-10]  江苏省卫健委9月10日发布，9月9日0-24时，江苏无新增本土确诊病例。新增境外输入确诊病例...   
69  [2021-09-11]  江苏省卫健委9月11日发布，9月10日0-24时，江苏无新增本土确诊病例。新增出院病例4例（...   
70  [2021-09-12]  江苏省卫健委9月12日发布，9月11日0-24时，江苏无新增本土确诊病例。新增境外输入无症状...   
71  [2021-09-13]  江苏省卫健委9月13日发布，9月12日0-24时，江苏无新增本土确诊病例。新增出院病例1例（...   
72  [2021-09-14]  江苏省卫健委9月14日发布，9月13日0-24时，江苏无新增本土确诊病例。新增出院病例3例（...   

                    name  name_label              age  age_label  \
0                     []           0     [岁, 岁, 岁